In [4]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=4da1025903d9d94943707c2ffd9bf7d9d2601446929608d91d1fe8492b6508c6
  Stored in directory: c:\users\steph\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [6]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Parse the html data:

In [7]:
soup = BeautifulSoup(html_data, "html.parser")
#print(soup.prettify())

Extract data from the table into three relevant columns:

In [8]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not Assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = ((str((row.span.text).split('(')[1:]).replace(' /',',')).replace(')',' ')).strip(' ')

#        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#print(table_contents)

Save the data to a dataframe:

In [9]:
df=pd.DataFrame(table_contents)

In [10]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(14)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,[]
1,M2A,Not assigned,[]
2,M3A,North York,['Parkwoods ']
3,M4A,North York,['Victoria Village ']
4,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']"
5,M6A,North York,"['Lawrence Manor, Lawrence Heights ']"
6,M7A,Queen's Park,['Ontario Provincial Government ']
7,M8A,Not assigned,[]
8,M9A,Etobicoke,['Islington Avenue ']
9,M1B,Scarborough,"['Malvern, Rouge ']"


Remove rows where borough not assigned:

In [11]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace=True)

df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,['Parkwoods ']
1,M4A,North York,['Victoria Village ']
2,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']"
3,M6A,North York,"['Lawrence Manor, Lawrence Heights ']"
4,M7A,Queen's Park,['Ontario Provincial Government ']
...,...,...,...
98,M8X,Etobicoke,"['The Kingsway, Montgomery Road, Old Mill Nort..."
99,M4Y,Downtown Toronto,['Church and Wellesley ']
100,M7Y,East Toronto Business,['Enclave of M4L ']
101,M8Y,Etobicoke,"[""Old Mill South, King's Mill Park, Sunnylea, ..."


(Still need to remove quotes and square brackets in Neighborhood series)

In [12]:
df.shape

(103, 3)

In [ ]:
#!pip install geocoder
# revisit this later

In [20]:
#postal_data = pd.read_csv('http://cocl.us/Geospatial_data') 
#postal_data.head()

# import csv from local machine
toronto_coords = pd.read_csv("C:\\Users\\steph\\pythonstuff\\Geospatial_Coordinates.csv")
toronto_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
df = pd.merge(df, toronto_coords, on='Postal Code')

Join the dataframe of neighborhoods with the latitude and longitude coordinates:

In [18]:
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,['Parkwoods '],43.753259,-79.329656
1,M4A,North York,['Victoria Village '],43.725882,-79.315572
2,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']",43.654260,-79.360636
3,M6A,North York,"['Lawrence Manor, Lawrence Heights ']",43.718518,-79.464763
4,M7A,Queen's Park,['Ontario Provincial Government '],43.662301,-79.389494
5,M9A,Etobicoke,['Islington Avenue '],43.667856,-79.532242
6,M1B,Scarborough,"['Malvern, Rouge ']",43.806686,-79.194353
7,M3B,North York,['Don Mills North'],43.745906,-79.352188
8,M4B,East York,"['Parkview Hill, Woodbine Gardens ']",43.706397,-79.309937
9,M5B,Downtown Toronto,"['Garden District, Ryerson ']",43.657162,-79.378937


In [19]:
df.shape

(103, 5)